In [143]:
from langchain_community.document_loaders import PyPDFLoader
pdfs=['SI.pdf','Placement.pdf']
documents=[]
for pdf in pdfs:
    loader=PyPDFLoader(pdf)
    document=loader.load()
    documents.extend(document)




In [144]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunked=text_splitter.split_documents(documents)


In [152]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()



db=FAISS.from_documents(chunked,GoogleGenerativeAIEmbeddings(google_api_key=os.getenv('Google_API_KEY'),model="models/embedding-001"))
db


In [153]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

template = """
Answer based on the context provided only .
I want answer from context only strictly but do use your AI skills. 
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)

In [154]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()
llm=ChatGoogleGenerativeAI(google_api_key=os.getenv('Google_API_KEY'),model="gemini-pro")
llm

ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001968906D670>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x0000019671E2EB80>, default_metadata=())

In [155]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import Runnable


doc_chain=create_stuff_documents_chain(llm,prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| PromptTemplate(input_variables=['context', 'input'], template='\nAnswer based on the context provided only .\nI want answer from context only strictly but do use your AI skills. \ncontext: {context}\ninput: {input}\nanswer:\n')
| ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001968906D670>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x0000019671E2EB80>, default_metadata=())
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [156]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001966EDD4F40>)

In [157]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,doc_chain)

In [160]:
response = retrieval_chain.invoke({"input": "Is DSA important, pls tell why, ans on basis of context only"})
response['answer']

'Yes, DSA is important as it gives you enough time to get comfortable with it and start competitive coding in general.'